In [1]:
import pandas as pd
import numpy as np

from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

import plotly.graph_objects as go

import src.data_loader as data_loader

In [2]:
DEFAULT_MARKER_SIZE = 9
HIGHLITED_MARKER_SIZE = 20
HIGHLITED_MARKER_COLOR = 'yellow'

## Tabulka Orgány

Některé orgány mají nadřazený orgán a pak je položka organy:organ_id_organ vyplněna, přičemž pouze v některých případech se tyto vazby využívají.

| Sloupec          | Typ      | Použití a vazby  |
|------------------|----------|------------------|
| id_organ         | int      | Identifikátor orgánu |
| organ_id_organ   | int      | Identifikátor nadřazeného orgánu, viz organy:id_organ |
| id_typ_organu    | int      | Typ orgánu, viz typ_organu:id_typ_organu |
| zkratka          | char(X)  | Zkratka orgánu, bez diakritiky, v některých připadech se zkratka při zobrazení nahrazuje jiným názvem |
| nazev_organu_cz  | char(X)  | Název orgánu v češtině |
| nazev_organu_en  | char(X)  | Název orgánu v angličtině |
| od_organ         | date  | Ustavení orgánu |
| do_organ         | date  | Ukončení orgánu |
| priorita         | int  | Priorita výpisu orgánů |
| cl_organ_base    | int  | Pokud je nastaveno na 1, pak při výpisu členů se nezobrazují záznamy v tabulkce zarazeni kde cl_funkce == 0. Toto chování odpovídá tomu, že v některých orgánech nejsou členové a teprve z nich se volí funkcionáři, ale přímo se volí do určité funkce. |

## Tabulka Poslanci

Uchovává další informace o poslanci vzhledem k volebnímu období: kontaktní údaje, adresa regionální kanceláře a podobně. Některé údaje jsou pouze v aktuálním volebním období.


| Sloupec          | Typ  | Použití a vazby |
|------------------|------|-----------------|
| id_poslanec | int | Identifikátor poslance |
| id_osoba | int | Identifikátor osoby, viz osoba:id_osoba |
| id_kraj | int | Volební kraj, viz organy:id_organu |
| id_kandidatka | int | Volební strana/hnutí, viz org:id_organu, pouze odkazuje na stranu/hnutí, za kterou byl zvolen a nemusí mít souvislost s členstvím v poslaneckém klubu. |
| id_obdobi | int | Volební období, viz organy:id_organu |
| web | char(X) | URL vlastních stránek poslance |
| ulice | char(X) | Adresa regionální kanceláře, ulice. |
| obec | char(X) | Adresa regionální kanceláře, obec. |
| psc | char(X) | Adresa regionální kanceláře, PSČ. |
| email | char(X) | E-mailová adresa poslance, případně obecná posta@psp.cz. |
| telefon | char(X) | Adresa regionální kanceláře, telefon. |
| fax | char(X) | Adresa regionální kanceláře, fax. |
| psp_telefon | char(X) | Telefonní číslo do kanceláře v budovách PS. |
| facebook | char(X) | URL stránky služby Facebook. |
| foto | int | Pokud je rovno 1, pak existuje fotografie poslance. |

## Tabulka Osoby

Obsahuje jména osob, které jsou zařazeni v orgánech. Vzhledem k tomu, že k jednoznačnému rozlišení osob často není dostatek informací, je možné, že ne všechny záznamy odkazují na jedinečné osoby, tj. některé osoby jsou v tabulce vícekrát.

| Sloupec | Typ | Použití a vazby |
|---------|-----|-----------------|
| id_osoba | int | Identifikátor osoby |
| pred | char(X) | Titul pred jmenem |
| jmeno | char(X) | Jméno
| prijmeni | char(X) | Příjmení, v některých případech obsahuje i dodatek typu "st.", "ml." |
| za | char(X) | Titul za jménem |
| narozeni | date | Datum narození, pokud neznámo, pak 1.1.1900. |
| pohlavi | char(X) | Pohlaví, "M" jako muž, ostatní hodnoty žena |
| zmena | date | Datum posledni změny |
| umrti | date | Datum úmrtí |

In [3]:
# process psp
data_loader.process_psp_files()
groups = data_loader.load_groups()
mps = data_loader.load_mps()
persons = data_loader.load_persons()

# process wikidata
data_loader.process_wikidata_files()
occupations = data_loader.load_occupations()

data = pd.merge(mps, groups[['id_organ', 'zkratka']], left_on='id_kandidatka', right_on='id_organ')
data = pd.merge(data, groups[['id_organ', 'od_organ', 'do_organ']], left_on='id_obdobi', right_on='id_organ')
data = data[['id_poslanec','id_osoba', 'zkratka', 'od_organ', 'do_organ']]
data = pd.merge(persons, data, on='id_osoba')
data = pd.merge(data, occupations, on='id_osoba')

data.head()

All files exist...
JSON already exists...


,id_osoba,pred,prijmeni,jmeno,za,narozeni,pohlavi,zmena,umrti,id_poslanec,zkratka,od_organ,do_organ,povolani
0,16,JUDr.,Buzková,Petra,,07.12.1965,Ž,07.12.1998,NaN,210,CSSD,01.06.1996,19.06.1998,[advokát]
1,16,JUDr.,Buzková,Petra,,07.12.1965,Ž,07.12.1998,NaN,436,CSSD,20.06.1998,20.06.2002,[advokát]
2,16,JUDr.,Buzková,Petra,,07.12.1965,Ž,07.12.1998,NaN,627,CSSD,15.06.2002,15.06.2006,[advokát]
3,21,MUDr.,Čermák,Petr,CSc.,22.01.1953,M,NaN,NaN,275,ODS,01.06.1996,19.06.1998,[lékař]
4,22,MVDr.,Černý,Jan,,23.04.1959,M,NaN,NaN,276,ODS,01.06.1996,19.06.1998,[veterinární lékař]


## Tabulka Hlasovani

Tabulka zaznamenává výsledek hlasování jednotlivého poslance.

V souborech uložena jako hlXXXXhN.unl, kde XXXX je reference volebního období a N je číslo části. V 6. a 7. volebním období obsahuje část č. 1 hlasování 1. až 50. schůze, část č. 2 hlasování od 51. schůze.

| Sloupec | Typ | Použití a vazby |
|---------|-----|-----------------|
| id_poslanec | int | Identifikátor poslance, viz poslanec:id_poslanec |
| id_hlasovani | int | Identifikátor hlasování, viz hl_hlasovani:id_hlasovani |
| vysledek | char(X) | Hlasování jednotlivého poslance. 'A' - ano, 'B' nebo 'N' - ne, 'C' - zdržel se (stiskl tlačítko X), 'F' - nehlasoval (byl přihlášen, ale nestiskl žádné tlačítko), '@' - nepřihlášen, 'M' - omluven, 'W' - hlasování před složením slibu poslance, 'K' - zdržel se/nehlasoval. Viz úvodní vysvětlení zpracování výsledků hlasování. |

In [4]:
votings = data_loader.load_votings()
votings.head()

,"('vysledek', 77302)_A","('vysledek', 77302)_X","('vysledek', 77303)_A","('vysledek', 77303)_X","('vysledek', 77304)_A","('vysledek', 77304)_X","('vysledek', 77305)_A","('vysledek', 77305)_X","('vysledek', 77306)_A","('vysledek', 77306)_X",...,"('vysledek', 78403)_A","('vysledek', 78403)_X","('vysledek', 78404)_A","('vysledek', 78404)_X","('vysledek', 78405)_A","('vysledek', 78405)_X","('vysledek', 78406)_A","('vysledek', 78406)_X","('vysledek', 78407)_A","('vysledek', 78407)_X"
id_poslanec,,,,,,,,,,,,,,,,,,,,,
1743,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
1744,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
1745,1,0,1,0,0,1,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
1746,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
1747,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,1,0,1,0


In [5]:
# first apply TruncatedSVD on data
svd_model = TruncatedSVD(n_components=50, n_iter=7)
votings_transformed = svd_model.fit_transform(votings)

# apply TSNE
tsne_model = TSNE(n_components=2, perplexity=20)
votings_transformed = tsne_model.fit_transform(votings_transformed)

votings_transformed = pd.DataFrame(votings_transformed)
votings_transformed['id_poslanec'] = votings.index
votings_transformed = votings_transformed.rename({0: 'x', 1: 'y'}, axis=1)

votings_transformed.head()

,x,y,id_poslanec
0,16.231369,10.026589,1743
1,-19.429920,-4.508680,1744
2,4.317335,5.428165,1745
3,-11.851210,-14.004259,1746
4,-15.983929,-11.987782,1747


In [6]:
data = pd.merge(votings_transformed, data, on='id_poslanec')
data = data[['id_osoba','jmeno','prijmeni','zkratka','povolani','x','y']]
data.head()

,id_osoba,jmeno,prijmeni,zkratka,povolani,x,y
0,5700,Ivan,Adamec,SPOLU,[pedagog],16.231369,10.026589
1,6254,Věra,Adámková,ANO2011,"[učitel, lékař, badatel]",-19.429920,-4.508680
2,6150,Andrej,Babiš,ANO2011,"[podnikatel, ekonom]",4.317335,5.428165
3,6428,Andrea,Babišová,ANO2011,[všeobecná sestra],-11.851210,-14.004259
4,6797,Vladimír,Balaš,PirSTAN,[pedagog],11.324593,14.537002


In [7]:
# prepare colors for each party
colors = ['red', 'blue', 'green', 'black', 'yellow', 'brown', 'gold', 'cyan', 'violet', 'orange']

parties_colors = {}
for ix, party in enumerate(data['zkratka'].unique()):
    parties_colors[party] = colors[ix]
    
parties_colors

{'SPOLU': 'red', 'ANO2011': 'blue', 'PirSTAN': 'green', 'SPD': 'black'}

In [8]:
print('\n'*3)

layout = go.Layout(
    xaxis = go.layout.XAxis(
        showticklabels=False),
    yaxis = go.layout.YAxis(
         showticklabels=False
    )
)

fig = go.Figure(layout=layout)

for ix, (zkratka, df) in enumerate(data.groupby('zkratka')):
    
    customdata = np.stack((df['jmeno'], df['prijmeni'], df['povolani'], df['zkratka']), axis=-1)
    
    fig.add_scatter(
        x=df['x'],
        y=df['y'],
        name=zkratka,
        mode='markers',
        customdata=customdata,
        hovertemplate="<br>".join([
            "Jméno: %{customdata[0]}",
            "Příjmení: %{customdata[1]}",
            "Povolání: %{customdata[2]}",
            "Strana: %{customdata[3]}"
        ]),
        marker_color=[parties_colors[zkratka] for _ in range(len(df))],
        marker_size=[DEFAULT_MARKER_SIZE for _ in range(len(df))],
        legendgroup=None
    )


fig.update_traces(showlegend=False).add_traces(
    [
        go.Scatter(name=k, x=[999999], y=[999999], marker_color=c, marker_size=DEFAULT_MARKER_SIZE, showlegend=True, mode='markers')
        for k, c in parties_colors.items()
    ]
)

fig.update_layout(yaxis_range=[data['y'].min() + data['y'].min() * 0.2, data['y'].max() + data['y'].max() * 0.2], xaxis_range=[data['x'].min() + data['x'].min() * 0.2, data['x'].max() + data['x'].max() * 0.2])


occupations = []

for tracer in fig["data"]:
    if tracer["customdata"] is not None:
        for mp in tracer["customdata"]:
            occupations.append(mp[2])
        
occupations = ["žádné"] + sorted(list(set([x for xs in occupations for x in xs])))

def get_occupation_colors(occupation):
    
    result = []
    
    for party_ix, party in enumerate(fig["data"]):
        if fig["data"][party_ix]['customdata'] is not None:
            x = [HIGHLITED_MARKER_COLOR if occupation in x[2] else fig["data"][party_ix]['marker']['color'][ix] for ix, x in enumerate(fig["data"][party_ix]['customdata'])]

            result.append( x )
        else:
            result.append( fig["data"][party_ix]['marker']['color'] )

    return result

def get_occupation_sizes(occupation):
    
    result = []
    
    for party_ix, party in enumerate(fig["data"]):
        if fig["data"][party_ix]['customdata'] is not None:
            x = [HIGHLITED_MARKER_SIZE if occupation in x[2] else DEFAULT_MARKER_SIZE for ix, x in enumerate(fig["data"][party_ix]['customdata'])]

            result.append( x )
        else:
            result.append( DEFAULT_MARKER_SIZE )

    return result

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": f"{occupation}",
                    "method": "update",
                    "args": [
                        {
                            "marker.size": get_occupation_sizes(occupation),
                             "marker.color": get_occupation_colors(occupation)
                        }                        
                    ],
                }
                for occupation in occupations
            ]
        }
    ],
    margin={"l": 0, "r": 0, "t": 25, "b": 0},
    height=700
)

fig.update_layout(
    title="Podobnost politiků při hlasování",
    title_x=0.5,
    title_y=0.999,
    legend_title="Strana",
    font=dict(
        family="Calibri",
        size=18
    )
)

fig.show()